# Solving nonlinear Equations in Julia

There are four types of nonlinear systems:

 1. The "standard nonlinear system", i.e. the `NonlinearProblem`. This is a system of
    equations with an initial condition where you want to satisfy all equations
    simultaneously.
 2. The "interval rootfinding problem", i.e. the `IntervalNonlinearProblem`. This is the
    case where you're given an interval `[a,b]` and need to find where `f(u) = 0` for `u`
    inside the bounds.
 3. The "steady state problem", i.e. find the `u` such that `u' = f(u) = 0`. While related
    to (1), it's not entirely the same because there's a uniquely defined privileged root.
 4. The nonlinear least squares problem, which is an under/over-constrained nonlinear system
    which might not be satisfiable, i.e. there may be no `u` such that `f(u) = 0`, and thus
    we find the `u` which minimizes `||f(u)||` in the least squares sense.

One important distinction is that (1) and (3) require the input and output sizes to be the
same, while (4) does not.

## Solving Nonlinear Systems of Equations

A nonlinear system $$f(u) = 0$$ is specified by defining a function `f(u,p)`, where `p` are
the parameters of the system. For example, the following solves the vector
equation $$f(u) = u^2 - p$$ for a vector of equations:

In [1]:
using NonlinearSolve

f(u, p) = u .* u .-p
u0 = [1.0, 1.0]
p = 2.0
prob = NonlinearProblem(f, u0, p)
sol = solve(prob)

retcode: Success
u: 2-element Vector{Float64}:
 1.4142135623730951
 1.4142135623730951

where `u0` is the initial condition for the rootfinder. Native NonlinearSolve.jl solvers use
the given type of `u0` to determine the type used within the solver and the return. Note
that the parameters `p` can be any type, but most are an AbstractArray for automatic
differentiation.

### Investigating the Solution

To investigate the solution, one can look at the elements of the `NonlinearSolution`. The
most important value is `sol.u`: this is the `u` that satisfies `f(u) = 0`. For example:

In [2]:
u = sol.u

2-element Vector{Float64}:
 1.4142135623730951
 1.4142135623730951

In [3]:
sol.resid

2-element Vector{Float64}:
 4.440892098500626e-16
 4.440892098500626e-16

In [4]:
sol.retcode

ReturnCode.Success = 1

There are multiple return codes which can mean the solve was successful, and thus we can use
the general command `SciMLBase.successful_retcode` to check whether the solution process
exited as intended:

In [5]:
SciMLBase.successful_retcode(sol)

true

In [6]:
sol.stats

SciMLBase.NLStats
Number of function evaluations:                    7
Number of Jacobians created:                       6
Number of factorizations:                          5
Number of linear solves:                           5
Number of nonlinear solver iterations:             5

### Interacting with the Solver Options

While `sol = solve(prob)` worked for our case here, in many situations you may need to interact more deeply with how the solving process is done. First things first, you can specify the solver using the positional arguments. For example, let's set the solver to `TrustRegion()`:

In [7]:
solve(prob, TrustRegion())

retcode: Success
u: 2-element Vector{Float64}:
 1.4142135623730951
 1.4142135623730951

For a complete list of solver choices, see [the nonlinear system solvers page](#https://docs.sciml.ai/NonlinearSolve/stable/solvers/nonlinear_system_solvers/#nonlinearsystemsolvers).

Next we can modify the tolerances. Here let's set some really low tolerances to force a tight solution:

In [8]:
solve(prob, TrustRegion(), reltol = 1e-12, abstol = 1e-12)

retcode: Success
u: 2-element Vector{Float64}:
 1.4142135623730951
 1.4142135623730951

## Solving Interval Rootfinding Problems with Bracketing Methods

For scalar rootfinding problems, bracketing methods exist in NonlinearSolve. The difference with bracketing methods is that with bracketing methods, instead of giving a `u0` initial condition, you pass a `uspan (a,b)` bracket in which the zero is expected to live. For example:

In [9]:
using NonlinearSolve

f(u, p) = u * u - 2.0
uspan = (1.0, 2.0) # brackets
prob_int = IntervalNonlinearProblem(f, uspan)
sol = solve(prob_int)

retcode: Success
u: 1.4142135623733618

## Solving Steady State Problems

For Steady State Problems, there is a wrapper package SteadyStateDiffEq.jl. This package automates handling SteadyStateProblems with NonlinearSolve and OrdinaryDiffEq.

In [14]:
using NonlinearSolve, SteadyStateDiffEq

f(u, p, t) = [2 - 2u[1]; u[1] - 4u[2]]
u0 = [0.0, 0.0]
prob = SteadyStateProblem(f, u0)

solve(prob, SSRootfind())

retcode: Success
u: 2-element Vector{Float64}:
 1.0
 0.25

In [23]:
function simpleODE!(du, u, p, t)
    a, b, c, f_source = p
    c_vec = @view u[:]
    du[1] = -a*c_vec[1] + b*c_vec[2] + f_source(c_vec[1])
    du[2] = -b*c_vec[2] - c*c_vec[3]
    du[3] = -c * c_vec[3] + a*c_vec[1]
end

simpleODE! (generic function with 1 method)

In [24]:
f_source(c) = c
u0 = [1., 1., 1.]
p = (0.1, 0.1, 0.1, f_source)
prob = SteadyStateProblem(simpleODE!, u0, p)

solve(prob, SSRootfind())

retcode: Success
u: 3-element Vector{Float64}:
 0.0
 0.0
 0.0

## Solving Nonlinear Least Squares Problems

In [15]:
using NonlinearSolve

function nlls!(du, u, p)
    du[1] = 2u[1] - 2
    du[2] = u[1] - 4u[2]
    du[3] = 0
end

nlls! (generic function with 1 method)

Note that here the output array is of length 3 while the input array is of length 2. We need to provide the `resid_prototype` to tell the solver what the output size is (this can be skipped for out of place problems):

In [16]:
u0 = [0.0, 0.0]
prob = NonlinearLeastSquaresProblem(
    NonlinearFunction(nlls!, resid_prototype = zeros(3)), u0)

solve(prob)

retcode: Success
u: 2-element Vector{Float64}:
 0.9999999999999998
 0.25